In [29]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [30]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

garbageCollectionDate[18]

'YarraRangesCouncil-garbage-collection-zones.json'

In [31]:
readInJson(garbageCollectionDate[18]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_b93ee6ec_bd97_4e4a_9146_7af63c8de0fb.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'MONDAY Area 1', 'rub_day': 'MONDAY',..."
1,Feature,ckan_b93ee6ec_bd97_4e4a_9146_7af63c8de0fb.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'FRIDAY Area 6', 'rub_day': 'FRIDAY',..."


In [32]:
df = readInJson(garbageCollectionDate[18])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [33]:
df.iloc[0,0]

{'name': 'MONDAY Area 1',
 'rub_day': 'MONDAY',
 'rub_weeks': 0,
 'rub_start': '2016-09-05',
 'rub_dates': None,
 'rub_scope': 'all (Service only for residentail)'}

In [35]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay == None:
        rub_colDay = ''
    elif rub_colFreq == None: 
        rub_colFreq = ''

    return rub_colFreq + rub_colDay.capitalize()

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))

df.drop('properties', axis = 1, inplace = True)

In [36]:
yarra = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay']]\
.sort_values(['locality', 'landfill_colDay'])\
        .drop_duplicates(['locality','landfill_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay'])\
.rename(columns = dict(zip(['landfill_colDay'], 
                           ['landfill'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

yarra.to_csv('../GBD_yarra.csv', index = False)